In [30]:
import torch
import torch.nn.functional as F

import numpy as np
import pandas as pd
import torchvision
import pretrainedmodels

from tqdm import tqdm
from torchvision import models
from torchvision import transforms
from torch.utils import model_zoo
from ipywidgets import FloatProgress
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from pathlib import Path

In [31]:
folder = Path("data/")
# the scrambled dataset
folder_scr = folder / "imagenette2_scr"
# the smoothened dataset
folder_smt = folder / "imagenette2_smt"
# the smoothened diff dataset
folder_smt_diff = folder / "imagenette2_smtdiff"
# the patched dataset
folder_ptd = folder / "imagenette2_ptd"

In [32]:
img_folders = [folder_scr, folder_smt, folder_smt_diff, folder_ptd]

Load the data with additional info to the dataset

In [33]:
imgn_data = pd.read_csv(folder / "imgnette_sample_data.csv", index_col=0)
imgn_data["path"] = imgn_data["path"].apply(lambda x: Path(x))
imgn_data[:3]

,filename,edges,edges_length,xmax_n,xmin_n,ymax_n,ymin_n,path,class,name,softm,inter_res_pos,inter_res_neg,res_perc,res_len,inter_stl_pos,inter_stl_neg,stl_perc,stl_len
order,,,,,,,,,,,,,,,,,,,
0,n01440764_11170,"[(32, 78), (32, 81), (32, 84), (32, 93), (32, ...",4302,158.0,32.0,183.0,77.0,/home/malte/Dokumente/Masterarbeit/data/imagen...,n01440764,"tench, Tinca tinca",0,1122.0,703.0,0.796077,5404.0,1814.0,228.0,0.532689,8076.0
1,n01440764_1009,"[(87, 73), (87, 75), (87, 98), (88, 73), (88, ...",212,165.0,87.0,108.0,55.0,/home/malte/Dokumente/Masterarbeit/data/imagen...,n01440764,"tench, Tinca tinca",0,0.0,30.0,0.101776,2083.0,9.0,0.0,0.034989,6059.0
2,n01440764_1091,"[(25, 58), (25, 150), (25, 151), (25, 211), (2...",1178,224.0,25.0,224.0,58.0,/home/malte/Dokumente/Masterarbeit/data/imagen...,n01440764,"tench, Tinca tinca",0,74.0,272.0,0.322122,3657.0,119.0,209.0,0.200716,5869.0


In [34]:
imgn_classes = pd.read_csv(folder / "imgn_classes.csv", index_col=0)
imgn_classes[:5]

,class,name,softm
0,n01440764,"tench, Tinca tinca",0
1,n01443537,"goldfish, Carassius auratus",1
2,n01484850,"great white shark, white shark, man-eater, man...",2
3,n01491361,"tiger shark, Galeocerdo cuvieri",3
4,n01494475,"hammerhead, hammerhead shark",4


In [35]:
transform = transforms.Compose([
 transforms.ToTensor()
])

In [36]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    https://gist.github.com/andrewjong/6b02ff237533b3b2c554701fb53d5c4d
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [37]:
def imgn_fn(file_path):
    for fln in imgn_data["filename"]:
        if fln in Path(file_path).stem:
            return fln

In [38]:
def pred_analysis(output, output_softm, imgn_classes):
    pred_acc, pred_class = torch.topk(output_softm, 1)
    pred_acc = pred_acc.squeeze().item()
    pred_class = pred_class.item()
    pred_logit = output[0][pred_class].item()
    pred_name = imgn_classes[imgn_classes.softm == pred_class].name.item()
    
    return pred_acc, pred_class, pred_logit, pred_name

In [39]:
def true_analysis(output, output_softm, imgn_fn, pt):
    # lookup filename to find original (unmodified) image and class
    orig_file = imgn_fn(pt[0])
    imgn_subs = imgn_data[imgn_data.filename == orig_file]
    
    true_class = imgn_subs.softm.item()
    true_logit = output[0][true_class].item()
    true_name = imgn_subs.name.item()
    true_acc = output_softm[0][true_class].item()
    
    return true_acc, true_class, true_logit, true_name, orig_file

In [40]:
def eval_model(model, data):
    eval_data = pd.DataFrame(columns=["file","orig_file",
                                      "pred_acc", "pred_class", "pred_logit", "pred_name",
                                     "true_acc", "true_class", "true_logit", "true_name"])
    
    for inp, cl, pt in tqdm(data):
        # run model on input image tensor
        output = model(inp)
        output_softm = F.softmax(output, dim=1)
        # get predicted (argmax) class values
        pred_acc, pred_class, pred_logit, pred_name = pred_analysis(output, output_softm, imgn_classes)
        # get true (real) class values
        true_acc, true_class, true_logit, true_name, orig_file = true_analysis(output, output_softm, imgn_fn, pt)
        
        # Pass a series in append() to append a row in dataframe
        eval_data = eval_data.append(pd.Series([Path(pt[0]).stem, orig_file, 
                                                pred_acc, pred_class, pred_logit, pred_name, 
                                               true_acc, true_class, true_logit, true_name], index=eval_data.columns ), ignore_index=True)
        
    return eval_data

____________________

## 1 Loading pretrained models

call `eval:model()` on each

In [41]:
datasets = list(map(lambda x: ImageFolderWithPaths(x, transform=transform), img_folders))
data = list(map(lambda x: DataLoader(x, batch_size=1, num_workers=2), datasets))

In [42]:
names = ["scr", "smt", "smt_diff", "ptd"]
data_names = list(zip(data, names))

In [43]:
print(data_names)

[(<torch.utils.data.dataloader.DataLoader object at 0x7f83e90030a0>, 'scr'), (<torch.utils.data.dataloader.DataLoader object at 0x7f83e87c6610>, 'smt'), (<torch.utils.data.dataloader.DataLoader object at 0x7f83e87c60a0>, 'smt_diff'), (<torch.utils.data.dataloader.DataLoader object at 0x7f83e8fe7e80>, 'ptd')]


In [44]:
def analysis_csv(model, data_names, model_name):
    for data, name in data_names:
        analysis = eval_model(model, data)
        analysis["model"] = model_name
        analysis.to_csv(folder / f"{name}_{model_name}.csv")

### VGG16

In [45]:
model = models.vgg16(pretrained=True)
model = model.eval()
model_name = "vgg16"

In [46]:
analysis_csv(model, data_names, model_name)

100%|██████████| 6060/6060 [31:35<00:00,  3.20it/s]


### ResNet 50

In [47]:
model = models.resnet50(pretrained=True)
model = model.eval()
model_name = "resnet50"

In [48]:
analysis_csv(model, data_names, model_name)

100%|██████████| 6060/6060 [14:03<00:00,  7.18it/s]


### SIN ResNet50

In [49]:
model = torchvision.models.resnet50(pretrained=False)
model = torch.nn.DataParallel(model)

checkpoint = torch.load('../../resnet50_train_60_epochs-c8e5653e.pth.tar', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["state_dict"])
model = model.eval()
model_name = "sin-resnet50"

In [50]:
analysis_csv(model, data_names, model_name)

100%|██████████| 6060/6060 [14:37<00:00,  6.91it/s]


### SE ResNet50

In [51]:
model = pretrainedmodels.se_resnet50(num_classes=1000, pretrained='imagenet')
model = model.eval()
model_name = "se-resnet50"

In [52]:
analysis_csv(model, data_names, model_name)

100%|██████████| 6060/6060 [14:50<00:00,  6.81it/s]


### ResNext 50

**Note**: `torchvision` version 0.5 is needed to load `ResNext`

In [53]:
model = torchvision.models.resnext50_32x4d(pretrained=True)
model = model.eval()
model_name = "resnext50"

In [54]:
analysis_csv(model, data_names, model_name)

100%|██████████| 6060/6060 [15:48<00:00,  6.39it/s]


### SE ResNext50

In [55]:
model = pretrainedmodels.se_resnext50_32x4d(num_classes=1000, pretrained='imagenet')
model = model.eval()
model_name = "se-resnext50"

In [56]:
analysis_csv(model, data_names, model_name)

100%|██████████| 6060/6060 [17:44<00:00,  5.69it/s]
